Dimensionality reduction
========

In [ ]:
import numpy as np
import pandas as pd
import glob
import os.path
import sys

# from svecon.HierarchicalGridSearchCV import HierarchicalGridSearchCV
from svecon.EmptyTransformer import EmptyTransformer

from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import Imputer, StandardScaler, LabelEncoder
from sklearn.decomposition import PCA
from sklearn.manifold import TSNE

from metric_learn import LMNN, NCA, LFDA, Covariance, CMAES, FullMatrixTransformer, NeuralNetworkTransformer
from metric_learn import ITML_Supervised, SDML_Supervised, LSML_Supervised, RCA_Supervised

import plotly
from plotly import tools
plotly.tools.set_credentials_file(username='sveco', api_key='8701ghzf0i')
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode()

datasetsDirectory = 'datasets'
resultsDirectory = 'datasets-results-dim-reduction'

if not os.path.exists(resultsDirectory):
    os.makedirs(resultsDirectory)

default_n_jobs = 8
default_random_state = 789
default_n_folds = 10
default_shuffle = True

import logging
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)
logger.handlers = []
fh = logging.FileHandler("{}/error.log".format(resultsDirectory))
fh.setLevel(logging.DEBUG)
logger.addHandler(fh)
fh = logging.StreamHandler(sys.stdout)
fh.setLevel(logging.ERROR)
logger.addHandler(fh)

defaultScatterMarker=dict(
    size=10,
    colorscale='Viridis',
    opacity=0.5
)

# np.set_printoptions(precision=7, suppress=True, threshold=np.nan)
np.set_printoptions(formatter={'float': lambda x: "{0:0.10f}".format(x)})

In [ ]:
import glob, os

datasets = []
for file in glob.glob("{}/*.csv".format(datasetsDirectory)):
    datasets.append(file)
datasets.sort()

# datasets.remove('datasets/soybean-large.csv')

# datasets = datasets[7:8]
logging.info("Datasets: " + str(datasets))

for x in datasets:
    print(x, pd.read_csv(x, sep=',', skiprows=1, header=0).shape)

In [ ]:
def makeScatter(Xy):
    X,y = Xy
    X = X.T
    if X.shape[0] in (0,1): return go.Scatter()
    
    return go.Scatter(x=X[0], y=X[1], #z=X_train_pca[2],
        text=y, mode='markers', marker={**defaultScatterMarker, 'color':y}, 
    )

import os.path
def loadOrCalculateAndSave(datasetName, X, y, method, params):
    fname = '{}/{}-{}-{}.csv'.format('precalculated-projections', datasetName, method.__name__, params)
    if os.path.isfile(fname):
        Xt = np.loadtxt(fname)
    else:
        try:
            Xt = method(**params).fit_transform(X, y)
        except:
            Xt = np.ndarray(shape=(1,2))
        np.savetxt(fname, Xt)
    return Xt, y

for filename in datasets:
    results = []
    datasetName = filename[len(datasetsDirectory)+1:-4]
    
    data = pd.read_csv(filename, sep=',', skiprows=1, header=0)
    y = data['class']
    X = data.drop(['class'], axis=1).values

    le = LabelEncoder()
    y = le.fit_transform(y)
    
    imputer = Imputer(missing_values='NaN', strategy='mean', axis=0, verbose=0, copy=False)
    X = imputer.fit_transform(X)
    
    trace1 = makeScatter(loadOrCalculateAndSave(datasetName, X, y, PCA, {'n_components':2}))
    trace2 = makeScatter(loadOrCalculateAndSave(datasetName, X, y, TSNE, {'n_components':2}))
    trace3 = makeScatter(loadOrCalculateAndSave(datasetName, X, y, LFDA, {'dim':2, 'k':2}))
    trace4 = makeScatter(loadOrCalculateAndSave(datasetName, X, y, CMAES, {'transformer':NeuralNetworkTransformer(layers=(6, 2,)), 'n_neighbors':4, 'n_gen':100}))

    fig = tools.make_subplots(
        rows=1,
        cols=4,
        print_grid=False,
        subplot_titles=('PCA', 'T-SNE', 'FDA', 'CMAES')
    )

    fig.append_trace(trace1, 1, 1)
    fig.append_trace(trace2, 1, 2)
    fig.append_trace(trace3, 1, 3)
    fig.append_trace(trace4, 1, 4)

    fig['layout'].update(title='{} dataset {}, {} classes'.format(datasetName, X.shape, len(np.unique(y))), showlegend=False)
#     fig['layout'].update(height=600, width=1200, title='i <3 subplots')
#     plot_url = py.plot(fig, filename='make-subplots')
    py.iplot(fig)

In [ ]:
# Authors: Fabian Pedregosa <fabian.pedregosa@inria.fr>
#          Olivier Grisel <olivier.grisel@ensta.org>
#          Mathieu Blondel <mathieu@mblondel.org>
#          Gael Varoquaux
# License: BSD 3 clause (C) INRIA 2011

print(__doc__)
from time import time

import numpy as np
import matplotlib.pyplot as plt
from matplotlib import offsetbox
from sklearn import (manifold, datasets, decomposition, ensemble,
                     discriminant_analysis, random_projection)

digits = datasets.load_digits(n_class=6)
X = digits.data
y = digits.target
n_samples, n_features = X.shape
n_neighbors = 30


#----------------------------------------------------------------------
# Scale and visualize the embedding vectors
def plot_embedding(X, title=None):
    x_min, x_max = np.min(X, 0), np.max(X, 0)
    X = (X - x_min) / (x_max - x_min)

    plt.figure()
    ax = plt.subplot(111)
    for i in range(X.shape[0]):
        plt.text(X[i, 0], X[i, 1], str(digits.target[i]),
                 color=plt.cm.Set1(y[i] / 10.),
                 fontdict={'weight': 'bold', 'size': 9})

#     if hasattr(offsetbox, 'AnnotationBbox'):
#         # only print thumbnails with matplotlib > 1.0
#         shown_images = np.array([[1., 1.]])  # just something big
#         for i in range(digits.data.shape[0]):
#             dist = np.sum((X[i] - shown_images) ** 2, 1)
#             if np.min(dist) < 4e-3:
#                 # don't show points that are too close
#                 continue
#             shown_images = np.r_[shown_images, [X[i]]]
#             imagebox = offsetbox.AnnotationBbox(
#                 offsetbox.OffsetImage(digits.images[i], cmap=plt.cm.gray_r),
#                 X[i])
#             ax.add_artist(imagebox)
    plt.xticks([]), plt.yticks([])
    if title is not None:
        plt.title(title)


#----------------------------------------------------------------------
# Plot images of the digits
n_img_per_row = 20
img = np.zeros((10 * n_img_per_row, 10 * n_img_per_row))
for i in range(n_img_per_row):
    ix = 10 * i + 1
    for j in range(n_img_per_row):
        iy = 10 * j + 1
        img[ix:ix + 8, iy:iy + 8] = X[i * n_img_per_row + j].reshape((8, 8))

plt.imshow(img, cmap=plt.cm.binary)
plt.xticks([])
plt.yticks([])
plt.title('A selection from the 64-dimensional digits dataset')


#----------------------------------------------------------------------
# Random 2D projection using a random unitary matrix
print("Computing random projection")
rp = random_projection.SparseRandomProjection(n_components=2, random_state=42)
X_projected = rp.fit_transform(X)
plot_embedding(X_projected, "Random Projection of the digits")


#----------------------------------------------------------------------
# Projection on to the first 2 principal components

print("Computing PCA projection")
t0 = time()
X_pca = decomposition.TruncatedSVD(n_components=2).fit_transform(X)
plot_embedding(X_pca,
               "Principal Components projection of the digits (time %.2fs)" %
               (time() - t0))

#----------------------------------------------------------------------
# Projection on to the first 2 linear discriminant components

print("Computing Linear Discriminant Analysis projection")
X2 = X.copy()
X2.flat[::X.shape[1] + 1] += 0.01  # Make X invertible
t0 = time()
X_lda = discriminant_analysis.LinearDiscriminantAnalysis(n_components=2).fit_transform(X2, y)
plot_embedding(X_lda,
               "Linear Discriminant projection of the digits (time %.2fs)" %
               (time() - t0))


#----------------------------------------------------------------------
# Isomap projection of the digits dataset
print("Computing Isomap embedding")
t0 = time()
X_iso = manifold.Isomap(n_neighbors, n_components=2).fit_transform(X)
print("Done.")
plot_embedding(X_iso,
               "Isomap projection of the digits (time %.2fs)" %
               (time() - t0))


#----------------------------------------------------------------------
# Locally linear embedding of the digits dataset
print("Computing LLE embedding")
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2,
                                      method='standard')
t0 = time()
X_lle = clf.fit_transform(X)
print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
plot_embedding(X_lle,
               "Locally Linear Embedding of the digits (time %.2fs)" %
               (time() - t0))


#----------------------------------------------------------------------
# Modified Locally linear embedding of the digits dataset
print("Computing modified LLE embedding")
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2,
                                      method='modified')
t0 = time()
X_mlle = clf.fit_transform(X)
print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
plot_embedding(X_mlle,
               "Modified Locally Linear Embedding of the digits (time %.2fs)" %
               (time() - t0))


#----------------------------------------------------------------------
# HLLE embedding of the digits dataset
print("Computing Hessian LLE embedding")
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2,
                                      method='hessian')
t0 = time()
X_hlle = clf.fit_transform(X)
print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
plot_embedding(X_hlle,
               "Hessian Locally Linear Embedding of the digits (time %.2fs)" %
               (time() - t0))


#----------------------------------------------------------------------
# LTSA embedding of the digits dataset
print("Computing LTSA embedding")
clf = manifold.LocallyLinearEmbedding(n_neighbors, n_components=2,
                                      method='ltsa')
t0 = time()
X_ltsa = clf.fit_transform(X)
print("Done. Reconstruction error: %g" % clf.reconstruction_error_)
plot_embedding(X_ltsa,
               "Local Tangent Space Alignment of the digits (time %.2fs)" %
               (time() - t0))

#----------------------------------------------------------------------
# MDS  embedding of the digits dataset
print("Computing MDS embedding")
clf = manifold.MDS(n_components=2, n_init=1, max_iter=100)
t0 = time()
X_mds = clf.fit_transform(X)
print("Done. Stress: %f" % clf.stress_)
plot_embedding(X_mds,
               "MDS embedding of the digits (time %.2fs)" %
               (time() - t0))

# ----------------------------------------------------------------------
# Random Trees embedding of the digits dataset
print("Computing Totally Random Trees embedding")
hasher = ensemble.RandomTreesEmbedding(n_estimators=200, random_state=0,
                                       max_depth=5)
t0 = time()
X_transformed = hasher.fit_transform(X)
pca = decomposition.TruncatedSVD(n_components=2)
X_reduced = pca.fit_transform(X_transformed)

plot_embedding(X_reduced,
               "Random forest embedding of the digits (time %.2fs)" %
               (time() - t0))

#----------------------------------------------------------------------
# Spectral embedding of the digits dataset
print("Computing Spectral embedding")
embedder = manifold.SpectralEmbedding(n_components=2, random_state=0,
                                      eigen_solver="arpack")
t0 = time()
X_se = embedder.fit_transform(X)

plot_embedding(X_se,
               "Spectral embedding of the digits (time %.2fs)" %
               (time() - t0))

#----------------------------------------------------------------------
# t-SNE embedding of the digits dataset
print("Computing t-SNE embedding")
tsne = manifold.TSNE(n_components=2, init='pca', random_state=0)
t0 = time()
X_tsne = tsne.fit_transform(X)
plot_embedding(X_tsne, "t-SNE embedding of the digits (time %.2fs)" % (time() - t0))

plt.show()

In [ ]:
#----------------------------------------------------------------------
# CMAES embedding of the digits dataset
print("Computing CMAES embedding")
cmaes = CMAES(dim=2, n_gen=100, n_neighbors=8, knn_weights='distance')
t0 = time()
X_cmaes = cmaes.fit_transform(X, y)
plot_embedding(X_cmaes, "CMAES embedding of the digits (time %.2fs)" % (time() - t0))
plt.show()

In [ ]:
cmaes.get_params()

In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from matplotlib import offsetbox

from sklearn.datasets import fetch_mldata, load_digits, fetch_olivetti_faces


# dataset = load_digits(n_class=10)
dataset = fetch_mldata('MNIST original')
dataset = fetch_olivetti_faces()
X = dataset.data
y = dataset.target

print(X.shape, y.shape)

# X = pd.DataFrame(X)
# X['class'] = pd.Series(y, index=X.index)

# print(X.shape)
# # X.to_csv('datasets/mnist.csv', index=False)